# 🔌 Week 07-08 · Notebook 16 · Model Context Protocol (MCP)

Learn Anthropic's MCP standard for connecting LLMs to external tools and data sources.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Understand what MCP is and why it matters for Gen AI applications
- Build a simple MCP server for manufacturing sensor data
- Connect an MCP server to an LLM client
- Compare MCP with traditional function calling and LangChain tools
- Implement MCP in a production manufacturing context

## 🧩 What is Model Context Protocol (MCP)?

**Model Context Protocol (MCP)** is an open standard created by Anthropic that defines how AI applications should communicate with external data sources and tools. Think of it as a **universal adapter** that allows LLMs to safely and efficiently access real-world data.

### Why MCP Matters

Before MCP, every tool integration was custom-built:
- ❌ Each application had its own way of connecting to databases
- ❌ No standard for how tools should expose their capabilities
- ❌ Security and permission management was ad-hoc
- ❌ Difficult to reuse tool integrations across different AI systems

With MCP:
- ✅ **Standardized**: One protocol for all tool integrations
- ✅ **Secure**: Built-in permission and authentication model
- ✅ **Reusable**: Write once, use in any MCP-compatible application
- ✅ **Composable**: Combine multiple MCP servers seamlessly

### MCP Architecture

```
┌─────────────────┐
│   LLM Client    │ (Claude, GPT, or any MCP-compatible client)
│  (Your App)     │
└────────┬────────┘
         │ MCP Protocol (JSON-RPC over stdio/HTTP)
         │
┌────────▼────────┐
│   MCP Server    │ (Exposes tools, resources, prompts)
│  (Python/Node)  │
└────────┬────────┘
         │
┌────────▼────────┐
│  Data Sources   │ (Databases, APIs, Files, Sensors)
│  & Tools        │
└─────────────────┘
```

### MCP vs. LangChain Tools

| Feature | MCP | LangChain Tools |
|---------|-----|----------------|
| **Standardization** | Open protocol | Framework-specific |
| **Reusability** | Works across any MCP client | LangChain only |
| **Security** | Built-in permission model | Manual implementation |
| **Language** | Language-agnostic | Python (primarily) |
| **Use Case** | Universal tool integration | Rapid prototyping |

**Bottom Line**: MCP is better for production systems, LangChain tools are faster for prototyping. **You can use both together!**

## 📦 Installation

Let's install the MCP Python SDK:

In [ ]:
!pip install mcp anthropic httpx -q

## 🏭 Use Case: Manufacturing Sensor MCP Server

We'll build an MCP server that exposes manufacturing sensor data to LLMs. This server will provide:
1. **Tools**: Functions to read sensor data, check equipment status
2. **Resources**: Access to historical sensor logs
3. **Prompts**: Pre-configured prompts for common diagnostics

## 🔧 Step 1: Create a Simple MCP Server

First, let's create a basic MCP server that exposes sensor reading functionality:

In [ ]:
# Save this to: mcp_sensor_server.py
# (In a real deployment, this would be a separate file)

from mcp.server import Server
from mcp.types import Tool, TextContent, ImageContent
import random
import json
from datetime import datetime

# Create MCP server instance
server = Server("manufacturing-sensor-server")

# Simulated sensor database
SENSORS_DB = {
    "CNC-A-102": {"type": "CNC Mill", "location": "Assembly Line A"},
    "PUMP-B-05": {"type": "Hydraulic Pump", "location": "Cooling System B"},
    "PRESS-C-12": {"type": "Stamping Press", "location": "Stamping Line C"},
}

# Define MCP tool for reading sensor data
@server.tool()
async def read_sensor_data(equipment_id: str) -> dict:
    """Read current sensor data for a specific piece of equipment.
    
    Args:
        equipment_id: The unique identifier for the equipment (e.g., 'CNC-A-102')
    
    Returns:
        Dictionary containing sensor readings (temperature, vibration, pressure)
    """
    if equipment_id not in SENSORS_DB:
        return {"error": f"Equipment {equipment_id} not found"}
    
    # Simulate sensor readings
    return {
        "equipment_id": equipment_id,
        "timestamp": datetime.now().isoformat(),
        "temperature_c": round(random.uniform(60.0, 95.0), 2),
        "vibration_mm_s": round(random.uniform(5.0, 15.0), 2),
        "pressure_bar": round(random.uniform(2.0, 5.0), 2),
        "status": "normal" if random.random() > 0.1 else "warning"
    }

@server.tool()
async def list_equipment() -> list:
    """List all monitored equipment in the facility.
    
    Returns:
        List of equipment with their details
    """
    return [
        {"id": eq_id, **details}
        for eq_id, details in SENSORS_DB.items()
    ]

@server.tool()
async def check_anomalies(equipment_id: str, threshold_vibration: float = 12.0) -> dict:
    """Check if equipment has anomalous sensor readings.
    
    Args:
        equipment_id: Equipment to check
        threshold_vibration: Vibration threshold in mm/s (default: 12.0)
    
    Returns:
        Analysis of whether readings are anomalous
    """
    data = await read_sensor_data(equipment_id)
    
    if "error" in data:
        return data
    
    anomalies = []
    if data["vibration_mm_s"] > threshold_vibration:
        anomalies.append(f"High vibration: {data['vibration_mm_s']} mm/s (threshold: {threshold_vibration})")
    if data["temperature_c"] > 90:
        anomalies.append(f"High temperature: {data['temperature_c']}°C")
    
    return {
        "equipment_id": equipment_id,
        "has_anomalies": len(anomalies) > 0,
        "anomalies": anomalies,
        "readings": data
    }

# Define MCP resource (read-only data)
@server.resource("sensor://historical/{equipment_id}")
async def get_historical_data(equipment_id: str) -> str:
    """Get historical sensor data for an equipment."""
    # In production, this would query a time-series database
    return f"Historical data for {equipment_id}:\n" + json.dumps({
        "last_24h_avg_temp": 78.5,
        "last_24h_avg_vibration": 8.2,
        "incident_count": 0
    }, indent=2)

# Define MCP prompt template
@server.prompt()
async def diagnose_equipment_prompt(equipment_id: str) -> str:
    """Generate a diagnostic prompt for equipment issues."""
    data = await read_sensor_data(equipment_id)
    return f"""Analyze the following sensor data for {equipment_id}:

{json.dumps(data, indent=2)}

Provide:
1. Assessment of equipment health
2. Potential issues or concerns
3. Recommended actions
4. Urgency level (low/medium/high)
"""

print("✅ MCP Sensor Server defined successfully")
print(f"Available tools: read_sensor_data, list_equipment, check_anomalies")
print(f"Available resources: sensor://historical/{{equipment_id}}")
print(f"Available prompts: diagnose_equipment_prompt")

## 🔗 Step 2: Connect to the MCP Server

Now let's create a client that connects to our MCP server and uses its tools:

In [ ]:
from mcp.client import Client
from anthropic import Anthropic
import os

# Initialize Anthropic client
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Initialize MCP client
async def use_mcp_server():
    """Example of using MCP server with Claude."""
    
    async with Client() as mcp_client:
        # Connect to our sensor server
        await mcp_client.connect_stdio("python", ["mcp_sensor_server.py"])
        
        # List available tools
        tools = await mcp_client.list_tools()
        print("📋 Available MCP Tools:")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
        
        # Use Claude with MCP tools
        messages = [
            {
                "role": "user",
                "content": "Check the sensor data for equipment CNC-A-102 and tell me if there are any issues."
            }
        ]
        
        # Claude will automatically call MCP tools as needed
        response = anthropic_client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            tools=[tool.to_params() for tool in tools],
            messages=messages
        )
        
        print("\n🤖 Claude's Response:")
        print(response.content[0].text)

# Run the example (uncomment when you have ANTHROPIC_API_KEY)
# await use_mcp_server()

## 🔄 Step 3: MCP with LangChain Integration

You can also use MCP servers with LangChain for the best of both worlds:

In [ ]:
from langchain.tools import StructuredTool
from langchain_anthropic import ChatAnthropic
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate

# Wrap MCP tools as LangChain tools
def wrap_mcp_tool_for_langchain(mcp_tool):
    """Convert an MCP tool to a LangChain tool."""
    async def tool_func(**kwargs):
        return await mcp_tool(**kwargs)
    
    return StructuredTool.from_function(
        func=tool_func,
        name=mcp_tool.__name__,
        description=mcp_tool.__doc__,
        coroutine=tool_func
    )

# Create LangChain-compatible tools from our MCP server
langchain_tools = [
    wrap_mcp_tool_for_langchain(read_sensor_data),
    wrap_mcp_tool_for_langchain(list_equipment),
    wrap_mcp_tool_for_langchain(check_anomalies),
]

# Create LangChain agent with MCP-backed tools
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a manufacturing diagnostic assistant. Use the available tools to check equipment status."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, langchain_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=langchain_tools, verbose=True)

# Use the agent
# result = agent_executor.invoke({
#     "input": "List all equipment and check for any anomalies"
# })
# print(result["output"])

## 🏭 Production Deployment Pattern

In production, you would deploy MCP servers as separate services:

```yaml
# docker-compose.yml
version: '3.8'
services:
  mcp-sensor-server:
    build: ./mcp-servers/sensor
    environment:
      - DATABASE_URL=postgresql://...
    ports:
      - "8001:8001"
  
  mcp-maintenance-server:
    build: ./mcp-servers/maintenance
    environment:
      - DATABASE_URL=postgresql://...
    ports:
      - "8002:8002"
  
  ai-app:
    build: ./app
    environment:
      - MCP_SENSOR_URL=http://mcp-sensor-server:8001
      - MCP_MAINTENANCE_URL=http://mcp-maintenance-server:8002
    depends_on:
      - mcp-sensor-server
      - mcp-maintenance-server
```

## 🎯 Key Takeaways

1. **MCP is a standard protocol** for connecting LLMs to external tools and data
2. **Better for production** than ad-hoc integrations due to standardization and security
3. **Can be used with LangChain** for maximum flexibility
4. **Three main components**: Tools (functions), Resources (data), Prompts (templates)
5. **Reusable**: One MCP server can serve multiple AI applications

### When to Use MCP vs. LangChain Tools:

**Use MCP when**:
- Building production systems
- Need to share tools across multiple applications
- Require strong security and permissions
- Working in a multi-language environment

**Use LangChain Tools when**:
- Rapid prototyping
- Staying within LangChain ecosystem
- Building Python-only applications
- Need quick iteration

**Best Practice**: Start with LangChain tools for prototyping, migrate to MCP for production! ✅

## 🔗 Additional Resources

- [MCP Official Documentation](https://modelcontextprotocol.io/)
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)
- [Anthropic MCP Guide](https://docs.anthropic.com/claude/docs/model-context-protocol)
- [MCP Server Examples](https://github.com/modelcontextprotocol/servers)

---

**Next Notebook**: Continue to `14_agent_state_management.ipynb` to learn about managing complex agent workflows! 🚀